### Loading liberary and dataset

In [11]:
library(ISLR2)
library(boot)

attach(Auto)

The following objects are masked from Auto (pos = 3):

    acceleration, cylinders, displacement, horsepower, mpg, name,
    origin, weight, year




### Validation Set Approach

In [2]:
set.seed(1) 
l=length(mpg) 
# pick l/2 from l, without put back
train = sample(l, l/2, replace = FALSE) 
# rest is validation is validation set
mpg.test = mpg[-train] 

#### fit 3 models respectively using the training data 

In [3]:
lm.fit1 = lm(mpg~horsepower,data=Auto,subset=train) 
lm.fit2 = lm(mpg~poly(horsepower,2),data=Auto,subset=train) 
lm.fit3 = lm(mpg~poly(horsepower,3),data=Auto,subset=train) 

#### make predictions for each model 

In [4]:
lm.pred1 = predict(lm.fit1,Auto[-train,]) 
lm.pred2 = predict(lm.fit2,Auto[-train,]) 
lm.pred3 = predict(lm.fit3,Auto[-train,]) 

#### calculate MSE for each model 

In [5]:
linear_MSE = mean((mpg.test-lm.pred1)^2) #linear 
qua_MSE = mean((mpg.test-lm.pred2)^2) #quadratic 
cub_MSE = mean((mpg.test-lm.pred3)^2) #cubic

In [6]:
linear_MSE
qua_MSE
cub_MSE

[1] 23.26601

[1] 18.71646

[1] 18.79401

### LOOCV

In [7]:
glm.fit = glm(mpg~horsepower,data=Auto) 
cv.err = cv.glm(Auto,glm.fit) 
cv.err$delta #average MSE and adjusted MSE

[1] 24.23151 24.23114

Adjusted mean squares are calculated by dividing the adjusted sum of squares by the degrees of freedom. 

The adjusted sum of squares does not depend on the order the factors are entered into the model. 


In [8]:
# Write loop statement to repeat LOOCV process for all models
cv.error = rep(0,5) #initial value

# for polynomials from order 1 to 5 calculate average MSE 
for (i in 1:5){ 
    glm.fit = glm(mpg~poly(horsepower,i),data=Auto)
    cv.error[i] = cv.glm(Auto,glm.fit)$delta[1]
}

cv.error

[1] 24.23151 19.24821 19.33498 19.42443 19.03321

In [9]:
# this code also could be done manually, instead of using cv.glm()

model_predict = array(0, dim = c(nrow(Auto),5))

for (i in 1:5) {
    for (index in 1:nrow(Auto)) {
        train = c(1:nrow(Auto))[-index]
        mpg.test = mpg[index] # rest is validation is validation set
        lm.fit = lm(mpg~poly(horsepower,i),data=Auto,subset=train) 
        lm.pred = predict(lm.fit,Auto[-train,]) 
        model_predict[index,i] =  lm.pred
    }
}

for (i in 1:5) {
    print(mean((model_predict[,i]-mpg)^2))
}

[1] 24.23151
[1] 19.24821
[1] 19.33498
[1] 19.42443
[1] 19.03321


### K-fold CV

In [10]:
set.seed(1) #test polynomials from order 1 to 10 (loop) 
cv.error.10 = rep(0,10)
for (i in 1:10) { 
    glm.fit = glm(mpg~poly(horsepower,i),data=Auto)
    cv.error.10[i] = cv.glm(Auto,glm.fit, K=10)$delta[1]
}

cv.error.10

[1] 24.21538 19.28327 19.12998 19.29201 19.09471 18.87874 18.78127 18.80484
 [9] 18.92676 18.99439